In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_planinfantil():
    """
    Función para hacer web scraping de la página https://www.planinfantil.es/planes/
    Extrae información sobre los planes infantiles disponibles
    """
    # URL de la página
    url = "https://www.planinfantil.es/planes/"
    
    # Headers para simular un navegador y evitar bloqueos
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'es-ES,es;q=0.9'
    }
    
    try:
        # Realizamos la petición HTTP
        response = requests.get(url, headers=headers)
        
        # Verificamos si la petición fue exitosa
        if response.status_code == 200:
            # Parseamos el contenido HTML con BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Lista para almacenar los planes
            planes = []
            
            # Inspeccionando la página real, ajustamos los selectores
            # Buscamos los planes en div con clase que contiene "plan-item" o similares
            articulos = soup.select('div.elementor-post')
            
            print(f"Se encontraron {len(articulos)} elementos que podrían ser planes")
            
            for articulo in articulos:
                try:
                    # Extraemos la información de cada plan con los selectores correctos
                    titulo = articulo.select_one('h3.elementor-post__title a').text.strip() if articulo.select_one('h3.elementor-post__title a') else "No disponible"
                    
                    # La descripción puede estar en varias clases, intentamos encontrarla
                    descripcion_elem = articulo.select_one('div.elementor-post__excerpt p')
                    descripcion = descripcion_elem.text.strip() if descripcion_elem else "No disponible"
                    
                    # Buscamos información de fecha y ubicación
                    meta_data = articulo.select('div.elementor-post__meta-data span')
                    fecha = meta_data[0].text.strip() if len(meta_data) > 0 else "No disponible"
                    ubicacion = meta_data[1].text.strip() if len(meta_data) > 1 else "No disponible"
                    
                    # Extraer la URL de la imagen
                    img_tag = articulo.select_one('div.elementor-post__thumbnail img')
                    imagen_url = img_tag['src'] if img_tag and 'src' in img_tag.attrs else "No disponible"
                    
                    # Extraer el enlace para más detalles
                    link_tag = articulo.select_one('a.elementor-post__thumbnail__link') or articulo.select_one('h3.elementor-post__title a')
                    enlace = link_tag['href'] if link_tag and 'href' in link_tag.attrs else "No disponible"
                    
                    # Imprimimos para debug
                    print(f"Plan encontrado: {titulo}")
                    
                    # Guardamos los datos en un diccionario
                    plan = {
                        'Título': titulo,
                        'Descripción': descripcion,
                        'Fecha': fecha,
                        'Ubicación': ubicacion,
                        'Imagen URL': imagen_url,
                        'Enlace': enlace
                    }
                    
                    planes.append(plan)
                except Exception as e:
                    print(f"Error al procesar un artículo: {str(e)}")
            
            # Creamos un DataFrame con la información extraída
            df = pd.DataFrame(planes)
            
            # Guardamos los datos en un archivo CSV
            df.to_csv('planes_infantiles.csv', index=False, encoding='utf-8-sig')
            
            print(f"Se han extraído {len(planes)} planes y se han guardado en 'planes_infantiles.csv'")
            return df
            
        else:
            print(f"Error al acceder a la página. Código de estado: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"Error durante el web scraping: {str(e)}")
        return None

# Alternativa usando otra aproximación más general para detectar los elementos
def scrape_planinfantil_alternativo():
    """
    Aproximación alternativa para extraer datos de la página
    """
    url = "https://www.planinfantil.es/planes/"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    }
    
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Guardamos el HTML para análisis
            with open("pagina_planes.html", "w", encoding="utf-8") as f:
                f.write(soup.prettify())
                
            print("HTML guardado en 'pagina_planes.html' para análisis manual")
            
            # Buscamos todos los enlaces que podrían llevar a planes
            enlaces = soup.select('a[href*="planinfantil.es/planes/"]')
            
            planes = []
            
            for enlace in enlaces:
                # Excluimos los enlaces de navegación, categorías, etc.
                if "category" in enlace['href'] or "page" in enlace['href']:
                    continue
                    
                titulo = enlace.text.strip()
                if not titulo:  # Si el enlace no tiene texto, buscar en elementos padre o hijos
                    titulo_elem = enlace.select_one('h3') or enlace.parent.select_one('h3')
                    if titulo_elem:
                        titulo = titulo_elem.text.strip()
                
                # Solo añadir si el título no está vacío y el enlace no es la misma página
                if titulo and enlace['href'] != url:
                    plan = {
                        'Título': titulo,
                        'Enlace': enlace['href']
                    }
                    planes.append(plan)
                    print(f"Plan encontrado: {titulo}")
            
            # Creamos un DataFrame con la información extraída
            df = pd.DataFrame(planes)
            
            if not df.empty:
                # Eliminamos duplicados basados en el enlace
                df = df.drop_duplicates(subset=['Enlace'])
                
                # Guardamos los datos en un archivo CSV
                df.to_csv('planes_infantiles_alt.csv', index=False, encoding='utf-8-sig')
                
                print(f"Se han extraído {len(df)} planes y se han guardado en 'planes_infantiles_alt.csv'")
                return df
            else:
                print("No se encontraron planes en la página")
                return None
        else:
            print(f"Error al acceder a la página. Código de estado: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"Error durante el web scraping: {str(e)}")
        return None

# Función para extraer información detallada de cada plan
def obtener_detalles_plan(url):
    """
    Función para extraer información detallada de un plan específico
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    }
    
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Extrae la información detallada del plan
            titulo = soup.select_one('h1.elementor-heading-title') or soup.select_one('h1')
            titulo = titulo.text.strip() if titulo else "No disponible"
            
            # Buscar la descripción en varios posibles contenedores
            descripcion_elem = soup.select_one('div.elementor-widget-container p') or soup.select_one('div.elementor-text-editor')
            descripcion_completa = descripcion_elem.text.strip() if descripcion_elem else "No disponible"
            
            # Intentar encontrar más información
            info_adicional = {}
            
            # Buscar todos los elementos que puedan contener información relevante
            elementos_info = soup.select('div.elementor-text-editor')
            
            for elem in elementos_info:
                texto = elem.text.strip()
                
                # Intentar identificar información relevante
                if "precio" in texto.lower():
                    info_adicional["Precio"] = texto
                elif "horario" in texto.lower():
                    info_adicional["Horario"] = texto
                elif "edad" in texto.lower():
                    info_adicional["Edad recomendada"] = texto
            
            detalles = {
                'Título': titulo,
                'Descripción completa': descripcion_completa,
                **info_adicional
            }
            
            return detalles
            
        else:
            print(f"Error al acceder a la página de detalles. Código de estado: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"Error al obtener detalles del plan: {str(e)}")
        return None

# Ejemplo de uso combinado
if __name__ == "__main__":
    # Intentamos primero con el método principal
    df_planes = scrape_planinfantil()
    
    # Si no funciona, intentamos con el método alternativo
    if df_planes is None or df_planes.empty:
        print("Intentando con método alternativo...")
        df_planes = scrape_planinfantil_alternativo()
    
    if df_planes is not None and not df_planes.empty:
        # Para cada plan, obtenemos los detalles
        detalles_planes = []
        
        for i, plan in df_planes.iterrows():
            titulo = plan.get('Título', f"Plan {i+1}")
            print(f"Obteniendo detalles del plan: {titulo}")
            
            # Solo procesamos los planes que tienen un enlace válido
            if 'Enlace' in plan and plan['Enlace'] != "No disponible":
                detalles = obtener_detalles_plan(plan['Enlace'])
                
                if detalles:
                    detalles_planes.append(detalles)
                    
                # Esperamos un poco entre peticiones para evitar sobrecarga
                time.sleep(2)
        
        # Guardamos los detalles en otro archivo CSV
        if detalles_planes:
            df_detalles = pd.DataFrame(detalles_planes)
            df_detalles.to_csv('detalles_planes_infantiles.csv', index=False, encoding='utf-8-sig')
            print(f"Se han extraído los detalles de {len(detalles_planes)} planes y se han guardado en 'detalles_planes_infantiles.csv'")

Se encontraron 0 elementos que podrían ser planes
Se han extraído 0 planes y se han guardado en 'planes_infantiles.csv'
Intentando con método alternativo...
HTML guardado en 'pagina_planes.html' para análisis manual
Plan encontrado: Teatro
Plan encontrado: Parques
Plan encontrado: Parques de Ocio
Plan encontrado: Rutas
Plan encontrado: Museos
Plan encontrado: PARQUES
Plan encontrado: TEATRO
Plan encontrado: PARQUES DE OCIO
Plan encontrado: MUSEOS
Plan encontrado: RUTAS
Plan encontrado: ESCAPADAS
Se han extraído 6 planes y se han guardado en 'planes_infantiles_alt.csv'
Obteniendo detalles del plan: Teatro
Obteniendo detalles del plan: Parques
Obteniendo detalles del plan: Parques de Ocio
Obteniendo detalles del plan: Rutas
Obteniendo detalles del plan: Museos
Obteniendo detalles del plan: ESCAPADAS
Se han extraído los detalles de 6 planes y se han guardado en 'detalles_planes_infantiles.csv'


In [5]:
detalles_planes

[{'Título': 'Teatro infantil de Madrid',
  'Descripción completa': 'No disponible'},
 {'Título': 'Parques infantiles de Madrid',
  'Descripción completa': 'No disponible'},
 {'Título': 'Parques de Ocio', 'Descripción completa': 'No disponible'},
 {'Título': 'Rutas', 'Descripción completa': 'No disponible'},
 {'Título': 'Museos', 'Descripción completa': 'No disponible'},
 {'Título': 'Escapadas', 'Descripción completa': 'No disponible'}]

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import uuid

def categorizar_evento(titulo, descripcion):
    """
    Función para categorizar un evento basado en su título y descripción
    """
    titulo_desc = (titulo + " " + descripcion).lower()
    
    if any(palabra in titulo_desc for palabra in ['taller', 'manualidad', 'crear', 'construir', 'artesanía']):
        return 'Talleres'
    elif any(palabra in titulo_desc for palabra in ['salud', 'bienestar', 'médico', 'terapia', 'hospital']):
        return 'Salud'
    elif any(palabra in titulo_desc for palabra in ['museo', 'teatro', 'concierto', 'exposición', 'arte', 'cultura', 'música']):
        return 'Cultural'
    elif any(palabra in titulo_desc for palabra in ['deporte', 'fútbol', 'baloncesto', 'natación', 'carrera', 'competición', 'olimpiada']):
        return 'Deportiva'
    else:
        return 'No especificada'

def determinar_discapacidad(titulo, descripcion):
    """
    Función para determinar si el evento está adaptado para alguna discapacidad
    """
    titulo_desc = (titulo + " " + descripcion).lower()
    
    if any(palabra in titulo_desc for palabra in ['visual', 'ciego', 'invidente', 'baja visión']):
        return 'Visual'
    elif any(palabra in titulo_desc for palabra in ['auditivo', 'sordo', 'hipoacusia']):
        return 'Auditiva'
    elif any(palabra in titulo_desc for palabra in ['motor', 'motriz', 'silla de ruedas', 'movilidad reducida']):
        return 'Motora'
    elif any(palabra in titulo_desc for palabra in ['accesible', 'adaptado', 'inclusiv', 'discapacidad']):
        return 'Adaptado - no especificada'
    else:
        return 'Ninguna'

def extraer_costo(texto):
    """
    Función para extraer el costo de un texto
    """
    # Patrones para buscar precios en formato español
    patrones = [
        r'(\d+[,.]?\d*)\s*€',  # 10 €, 10,50 €
        r'(\d+[,.]?\d*)\s*euros',  # 10 euros, 10,50 euros
        r'precio[:\s]*(\d+[,.]?\d*)',  # precio: 10, precio 10,50
        r'cuesta[:\s]*(\d+[,.]?\d*)',  # cuesta: 10, cuesta 10,50
    ]
    
    texto_lower = texto.lower()
    
    if 'gratis' in texto_lower or 'gratuito' in texto_lower or 'libre' in texto_lower:
        return '0 €'
    
    for patron in patrones:
        match = re.search(patron, texto_lower)
        if match:
            return f"{match.group(1)} €"
    
    return 'No especificado'

def determinar_edad(texto):
    """
    Función para determinar la edad dirigida
    """
    # Patrones para buscar edades
    patrones = [
        r'(\d+)[-\s]*(\d+)\s*años',  # 3-12 años, 3 a 12 años
        r'de\s*(\d+)\s*a\s*(\d+)\s*años',  # de 3 a 12 años
        r'edad[:\s]*(\d+)[-\s]*(\d+)',  # edad: 3-12, edad 3-12
        r'(\d+)\+\s*años',  # 3+ años
        r'mayores de (\d+)',  # mayores de 3
        r'a partir de (\d+)',  # a partir de 3
    ]
    
    texto_lower = texto.lower()
    
    for patron in patrones:
        match = re.search(patron, texto_lower)
        if match:
            if len(match.groups()) == 2:
                return f"{match.group(1)}-{match.group(2)} años"
            else:
                return f"{match.group(1)}+ años"
    
    # Verificar términos comunes
    if any(term in texto_lower for term in ['infantil', 'niños', 'pequeños']):
        return '3-12 años'
    elif any(term in texto_lower for term in ['adolescente', 'jóvenes', 'juvenil']):
        return '12-18 años'
    elif any(term in texto_lower for term in ['familiar', 'familia']):
        return 'Todas las edades'
    
    return 'No especificado'

def determinar_modalidad(texto):
    """
    Función para determinar si el evento es en interior o exterior
    """
    texto_lower = texto.lower()
    
    interior = ['interior', 'sala', 'aula', 'museo', 'teatro', 'centro', 'edificio']
    exterior = ['exterior', 'parque', 'aire libre', 'jardín', 'piscina', 'playa', 'montaña', 'naturaleza']
    
    if any(palabra in texto_lower for palabra in exterior):
        return 'Exterior'
    elif any(palabra in texto_lower for palabra in interior):
        return 'Interior'
    else:
        return 'No especificado'

def determinar_min_integrantes(texto):
    """
    Función para determinar el mínimo de integrantes
    """
    patrones = [
        r'mínimo\s*(\d+)\s*persona',  # mínimo 5 personas
        r'grupo[s]?\s*de\s*(\d+)',  # grupos de 5
        r'(\d+)\s*participante[s]?\s*mínimo',  # 5 participantes mínimo
    ]
    
    texto_lower = texto.lower()
    
    for patron in patrones:
        match = re.search(patron, texto_lower)
        if match:
            return match.group(1)
    
    if 'individual' in texto_lower or 'personal' in texto_lower:
        return '1'
    elif 'grupo' in texto_lower or 'equipo' in texto_lower:
        return '2'
    
    return 'No especificado'

def scrape_planinfantil_estructurado():
    """
    Función para hacer web scraping de la página https://www.planinfantil.es/planes/
    y estructurar los datos según lo requerido
    """
    # URL de la página
    url = "https://www.planinfantil.es/planes/"
    
    # Headers para simular un navegador y evitar bloqueos
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'es-ES,es;q=0.9'
    }
    
    try:
        # Realizamos la petición HTTP
        response = requests.get(url, headers=headers)
        
        # Verificamos si la petición fue exitosa
        if response.status_code == 200:
            # Parseamos el contenido HTML con BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Lista para almacenar los planes con la estructura requerida
            planes_estructurados = []
            
            # Buscamos los planes en div con clase que contiene "plan-item" o similares
            articulos = soup.select('div.elementor-post')
            
            print(f"Se encontraron {len(articulos)} elementos que podrían ser planes")
            
            contador = 1
            for articulo in articulos:
                try:
                    # Generamos un ID único
                    id_evento = str(uuid.uuid4())[:8]
                    
                    # Extraemos la información básica
                    titulo_elem = articulo.select_one('h3.elementor-post__title a')
                    titulo = titulo_elem.text.strip() if titulo_elem else "No disponible"
                    
                    descripcion_elem = articulo.select_one('div.elementor-post__excerpt p')
                    descripcion = descripcion_elem.text.strip() if descripcion_elem else "No disponible"
                    
                    # Combinamos título y descripción para análisis
                    texto_completo = f"{titulo} {descripcion}"
                    
                    # Extraemos o inferimos el resto de la información
                    meta_data = articulo.select('div.elementor-post__meta-data span')
                    ubicacion = meta_data[1].text.strip() if len(meta_data) > 1 else "No especificado"
                    
                    # Categorización según el contenido
                    categoria = categorizar_evento(titulo, descripcion)
                    discapacidad = determinar_discapacidad(titulo, descripcion)
                    costo = extraer_costo(texto_completo)
                    edad_dirigida = determinar_edad(texto_completo)
                    modalidad = determinar_modalidad(texto_completo)
                    min_integrantes = determinar_min_integrantes(texto_completo)
                    
                    # Guardamos los datos en un diccionario con la estructura requerida
                    plan_estructurado = {
                        'id_evento': id_evento,
                        'nombre_evento': titulo,
                        'categoria': categoria,
                        'discapacidad': discapacidad,
                        'ubicacion': ubicacion,
                        'costo': costo,
                        'edad_dirigida': edad_dirigida,
                        'min_integrantes': min_integrantes,
                        'modalidad': modalidad
                    }
                    
                    planes_estructurados.append(plan_estructurado)
                    print(f"Plan {contador} procesado: {titulo}")
                    contador += 1
                    
                except Exception as e:
                    print(f"Error al procesar un artículo: {str(e)}")
            
            # Creamos un DataFrame con la información estructurada
            df = pd.DataFrame(planes_estructurados)
            
            # Guardamos los datos en un archivo CSV
            df.to_csv('planes_infantiles_estructurados.csv', index=False, encoding='utf-8-sig')
            
            print(f"Se han extraído y estructurado {len(planes_estructurados)} planes y se han guardado en 'planes_infantiles_estructurados.csv'")
            return df
            
        else:
            print(f"Error al acceder a la página. Código de estado: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"Error durante el web scraping: {str(e)}")
        return None

# Versión alternativa que obtiene más detalles navegando a cada página de evento
def scrape_planinfantil_estructurado_detallado():
    """
    Función para hacer web scraping más detallado, visitando cada página de evento
    """
    # URL de la página
    url = "https://www.planinfantil.es/planes/"
    
    # Headers para simular un navegador
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    }
    
    try:
        # Realizamos la petición HTTP
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Lista para almacenar los planes con la estructura requerida
            planes_estructurados = []
            
            # Buscamos los enlaces a las páginas de eventos
            articulos = soup.select('div.elementor-post')
            
            print(f"Se encontraron {len(articulos)} elementos que podrían ser planes")
            
            contador = 1
            for articulo in articulos:
                try:
                    # Extraemos información básica
                    titulo_elem = articulo.select_one('h3.elementor-post__title a')
                    nombre_evento = titulo_elem.text.strip() if titulo_elem else "No disponible"
                    
                    # Extraer el enlace para más detalles
                    link_tag = articulo.select_one('a.elementor-post__thumbnail__link') or articulo.select_one('h3.elementor-post__title a')
                    enlace = link_tag['href'] if link_tag and 'href' in link_tag.attrs else None
                    
                    # Si no hay enlace, continuamos con el siguiente
                    if not enlace:
                        print(f"No se encontró enlace para: {nombre_evento}")
                        continue
                    
                    print(f"Obteniendo detalles de: {nombre_evento} ({enlace})")
                    
                    # Visitamos la página del evento para obtener más detalles
                    try:
                        detalle_response = requests.get(enlace, headers=headers)
                        if detalle_response.status_code == 200:
                            detalle_soup = BeautifulSoup(detalle_response.content, 'html.parser')
                            
                            # Extraemos toda la información textual de la página
                            texto_completo = detalle_soup.get_text()
                            
                            # Generamos un ID único
                            id_evento = str(uuid.uuid4())[:8]
                            
                            # Meta datos
                            meta_data = articulo.select('div.elementor-post__meta-data span')
                            ubicacion = meta_data[1].text.strip() if len(meta_data) > 1 else "No especificado"
                            
                            # Analizamos el texto para extraer/inferir información
                            categoria = categorizar_evento(nombre_evento, texto_completo)
                            discapacidad = determinar_discapacidad(nombre_evento, texto_completo)
                            costo = extraer_costo(texto_completo)
                            edad_dirigida = determinar_edad(texto_completo)
                            modalidad = determinar_modalidad(texto_completo)
                            min_integrantes = determinar_min_integrantes(texto_completo)
                            
                            # Guardamos los datos en un diccionario con la estructura requerida
                            plan_estructurado = {
                                'id_evento': id_evento,
                                'nombre_evento': nombre_evento,
                                'categoria': categoria,
                                'discapacidad': discapacidad,
                                'ubicacion': ubicacion,
                                'costo': costo,
                                'edad_dirigida': edad_dirigida,
                                'min_integrantes': min_integrantes,
                                'modalidad': modalidad
                            }
                            
                            planes_estructurados.append(plan_estructurado)
                            print(f"Plan {contador} procesado: {nombre_evento}")
                            
                        else:
                            print(f"Error al acceder a la página del evento: {enlace}")
                    except Exception as e:
                        print(f"Error al obtener detalles de {nombre_evento}: {str(e)}")
                    
                    # Esperamos para no sobrecargar el servidor
                    time.sleep(2)
                    contador += 1
                    
                except Exception as e:
                    print(f"Error al procesar un artículo: {str(e)}")
            
            # Creamos un DataFrame con la información estructurada
            df = pd.DataFrame(planes_estructurados)
            
            # Guardamos los datos en un archivo CSV
            df.to_csv('planes_infantiles_detallados.csv', index=False, encoding='utf-8-sig')
            
            print(f"Se han extraído y estructurado {len(planes_estructurados)} planes detallados y se han guardado en 'planes_infantiles_detallados.csv'")
            return df
            
        else:
            print(f"Error al acceder a la página. Código de estado: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"Error durante el web scraping: {str(e)}")
        return None

# Ejecutar el código
if __name__ == "__main__":
    print("Iniciando web scraping básico...")
    df_basico = scrape_planinfantil_estructurado()
    
    if df_basico is None or df_basico.empty:
        print("\nRealizando web scraping detallado...")
        df_detallado = scrape_planinfantil_estructurado_detallado()

Iniciando web scraping básico...
Se encontraron 0 elementos que podrían ser planes
Se han extraído y estructurado 0 planes y se han guardado en 'planes_infantiles_estructurados.csv'

Realizando web scraping detallado...
Se encontraron 0 elementos que podrían ser planes
Se han extraído y estructurado 0 planes detallados y se han guardado en 'planes_infantiles_detallados.csv'


In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import uuid

def inicializar_driver():
    """
    Inicializa y configura el driver de Selenium
    """
    # Configuramos opciones para Chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Ejecutar en modo headless (sin interfaz gráfica)
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    # Inicializamos el driver
    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        return driver
    except Exception as e:
        print(f"Error al inicializar el driver: {str(e)}")
        
        # Alternativa si ChromeDriverManager falla
        try:
            driver = webdriver.Chrome(options=chrome_options)
            return driver
        except Exception as e2:
            print(f"Error alternativo: {str(e2)}")
            return None

def categorizar_evento(titulo, descripcion):
    """
    Función para categorizar un evento basado en su título y descripción
    """
    titulo_desc = (titulo + " " + descripcion).lower()
    
    if any(palabra in titulo_desc for palabra in ['taller', 'manualidad', 'crear', 'construir', 'artesanía']):
        return 'Talleres'
    elif any(palabra in titulo_desc for palabra in ['salud', 'bienestar', 'médico', 'terapia', 'hospital']):
        return 'Salud'
    elif any(palabra in titulo_desc for palabra in ['museo', 'teatro', 'concierto', 'exposición', 'arte', 'cultura', 'música']):
        return 'Cultural'
    elif any(palabra in titulo_desc for palabra in ['deporte', 'fútbol', 'baloncesto', 'natación', 'carrera', 'competición', 'olimpiada']):
        return 'Deportiva'
    else:
        return 'No especificada'

def determinar_discapacidad(titulo, descripcion):
    """
    Función para determinar si el evento está adaptado para alguna discapacidad
    """
    titulo_desc = (titulo + " " + descripcion).lower()
    
    if any(palabra in titulo_desc for palabra in ['visual', 'ciego', 'invidente', 'baja visión']):
        return 'Visual'
    elif any(palabra in titulo_desc for palabra in ['auditivo', 'sordo', 'hipoacusia']):
        return 'Auditiva'
    elif any(palabra in titulo_desc for palabra in ['motor', 'motriz', 'silla de ruedas', 'movilidad reducida']):
        return 'Motora'
    elif any(palabra in titulo_desc for palabra in ['accesible', 'adaptado', 'inclusiv', 'discapacidad']):
        return 'Adaptado - no especificada'
    else:
        return 'Ninguna'

def extraer_costo(texto):
    """
    Función para extraer el costo de un texto
    """
    # Patrones para buscar precios en formato español
    patrones = [
        r'(\d+[,.]?\d*)\s*€',  # 10 €, 10,50 €
        r'(\d+[,.]?\d*)\s*euros',  # 10 euros, 10,50 euros
        r'precio[:\s]*(\d+[,.]?\d*)',  # precio: 10, precio 10,50
        r'cuesta[:\s]*(\d+[,.]?\d*)',  # cuesta: 10, cuesta 10,50
    ]
    
    texto_lower = texto.lower()
    
    if 'gratis' in texto_lower or 'gratuito' in texto_lower or 'libre' in texto_lower:
        return '0 €'
    
    for patron in patrones:
        match = re.search(patron, texto_lower)
        if match:
            return f"{match.group(1)} €"
    
    return 'No especificado'

def determinar_edad(texto):
    """
    Función para determinar la edad dirigida
    """
    # Patrones para buscar edades
    patrones = [
        r'(\d+)[-\s]*(\d+)\s*años',  # 3-12 años, 3 a 12 años
        r'de\s*(\d+)\s*a\s*(\d+)\s*años',  # de 3 a 12 años
        r'edad[:\s]*(\d+)[-\s]*(\d+)',  # edad: 3-12, edad 3-12
        r'(\d+)\+\s*años',  # 3+ años
        r'mayores de (\d+)',  # mayores de 3
        r'a partir de (\d+)',  # a partir de 3
    ]
    
    texto_lower = texto.lower()
    
    for patron in patrones:
        match = re.search(patron, texto_lower)
        if match:
            if len(match.groups()) == 2:
                return f"{match.group(1)}-{match.group(2)} años"
            else:
                return f"{match.group(1)}+ años"
    
    # Verificar términos comunes
    if any(term in texto_lower for term in ['infantil', 'niños', 'pequeños']):
        return '3-12 años'
    elif any(term in texto_lower for term in ['adolescente', 'jóvenes', 'juvenil']):
        return '12-18 años'
    elif any(term in texto_lower for term in ['familiar', 'familia']):
        return 'Todas las edades'
    
    return 'No especificado'

def determinar_modalidad(texto):
    """
    Función para determinar si el evento es en interior o exterior
    """
    texto_lower = texto.lower()
    
    interior = ['interior', 'sala', 'aula', 'museo', 'teatro', 'centro', 'edificio']
    exterior = ['exterior', 'parque', 'aire libre', 'jardín', 'piscina', 'playa', 'montaña', 'naturaleza']
    
    if any(palabra in texto_lower for palabra in exterior):
        return 'Exterior'
    elif any(palabra in texto_lower for palabra in interior):
        return 'Interior'
    else:
        return 'No especificado'

def determinar_min_integrantes(texto):
    """
    Función para determinar el mínimo de integrantes
    """
    patrones = [
        r'mínimo\s*(\d+)\s*persona',  # mínimo 5 personas
        r'grupo[s]?\s*de\s*(\d+)',  # grupos de 5
        r'(\d+)\s*participante[s]?\s*mínimo',  # 5 participantes mínimo
    ]
    
    texto_lower = texto.lower()
    
    for patron in patrones:
        match = re.search(patron, texto_lower)
        if match:
            return match.group(1)
    
    if 'individual' in texto_lower or 'personal' in texto_lower:
        return '1'
    elif 'grupo' in texto_lower or 'equipo' in texto_lower:
        return '2'
    
    return 'No especificado'

def scrape_planinfantil_selenium():
    """
    Función para hacer web scraping de la página https://www.planinfantil.es/planes/
    utilizando Selenium para cargar contenido dinámico
    """
    # URL de la página
    url = "https://www.planinfantil.es/planes/"
    
    # Inicializamos el driver
    driver = inicializar_driver()
    if not driver:
        print("No se pudo inicializar el driver de Selenium. ¿Está instalado Chrome?")
        return None
    
    try:
        # Abrimos la página
        print(f"Accediendo a {url}...")
        driver.get(url)
        
        # Esperamos a que la página cargue completamente
        time.sleep(5)  # Damos tiempo para que cargue el contenido dinámico
        
        # Imprimimos el título de la página para verificar que se cargó correctamente
        print(f"Título de la página: {driver.title}")
        
        # Guardamos el HTML para análisis manual si es necesario
        with open('pagina_selenium.html', 'w', encoding='utf-8') as f:
            f.write(driver.page_source)
        print("HTML guardado en 'pagina_selenium.html'")
        
        # Analizamos la estructura de la página
        print("Buscando eventos en la página...")
        
        # Lista para almacenar la información de los eventos
        eventos = []
        
        # Intentamos diferentes selectores para encontrar los eventos
        selectores = [
            'article', 
            '.elementor-posts-container article',
            '.elementor-post',
            '.elementor-posts article',
            '.elementor-post__card',
            '.elementor-grid article',
            '.elementor-grid-item'
        ]
        
        elementos_encontrados = False
        
        for selector in selectores:
            try:
                elementos = driver.find_elements(By.CSS_SELECTOR, selector)
                if elementos:
                    print(f"Se encontraron {len(elementos)} elementos con el selector '{selector}'")
                    elementos_encontrados = True
                    
                    for i, elemento in enumerate(elementos):
                        try:
                            # Intentamos extraer el título, puede estar en diferentes elementos
                            titulo = None
                            for selector_titulo in ['h3', 'h2', 'h1', '.elementor-post__title']:
                                try:
                                    titulo_elem = elemento.find_element(By.CSS_SELECTOR, selector_titulo)
                                    titulo = titulo_elem.text.strip()
                                    if titulo:
                                        break
                                except:
                                    continue
                            
                            if not titulo:
                                continue  # Si no encontramos un título, pasamos al siguiente elemento
                            
                            # Intentamos extraer la descripción, puede estar en diferentes elementos
                            descripcion = "No disponible"
                            for selector_desc in ['.elementor-post__excerpt', 'p', '.elementor-post__text']:
                                try:
                                    desc_elem = elemento.find_element(By.CSS_SELECTOR, selector_desc)
                                    descripcion = desc_elem.text.strip()
                                    if descripcion:
                                        break
                                except:
                                    continue
                            
                            # Intentamos conseguir la URL del evento para visitarla luego
                            url_evento = None
                            try:
                                link = elemento.find_element(By.TAG_NAME, 'a')
                                url_evento = link.get_attribute('href')
                            except:
                                url_evento = None
                            
                            # Imprimimos para debug
                            print(f"Evento {i+1}: {titulo} - URL: {url_evento}")
                            
                            # Generamos un ID único
                            id_evento = str(uuid.uuid4())[:8]
                            
                            # Extraemos o inferimos el resto de la información
                            texto_completo = f"{titulo} {descripcion}"
                            
                            # Categorización según el contenido
                            categoria = categorizar_evento(titulo, descripcion)
                            discapacidad = determinar_discapacidad(titulo, descripcion)
                            costo = extraer_costo(texto_completo)
                            edad_dirigida = determinar_edad(texto_completo)
                            modalidad = determinar_modalidad(texto_completo)
                            min_integrantes = determinar_min_integrantes(texto_completo)
                            
                            # Extraemos la ubicación o usamos valor por defecto
                            ubicacion = "Madrid"  # Valor por defecto
                            for selector_ubicacion in ['.elementor-post__meta-data', '.location', '.ubicacion']:
                                try:
                                    ubicacion_elem = elemento.find_element(By.CSS_SELECTOR, selector_ubicacion)
                                    ubicacion = ubicacion_elem.text.strip()
                                    break
                                except:
                                    continue
                            
                            # Si tenemos la URL del evento, visitamos la página para obtener más detalles
                            if url_evento:
                                try:
                                    # Abrimos la página del evento en una nueva pestaña
                                    driver.execute_script(f"window.open('{url_evento}', '_blank');")
                                    driver.switch_to.window(driver.window_handles[1])
                                    
                                    # Esperamos a que cargue
                                    time.sleep(3)
                                    
                                    # Obtenemos todo el texto de la página
                                    texto_detallado = driver.find_element(By.TAG_NAME, 'body').text
                                    
                                    # Actualizamos datos con información más detallada
                                    categoria = categorizar_evento(titulo, texto_detallado)
                                    discapacidad = determinar_discapacidad(titulo, texto_detallado)
                                    costo = extraer_costo(texto_detallado)
                                    edad_dirigida = determinar_edad(texto_detallado)
                                    modalidad = determinar_modalidad(texto_detallado)
                                    min_integrantes = determinar_min_integrantes(texto_detallado)
                                    
                                    # Cerramos la pestaña y volvemos a la principal
                                    driver.close()
                                    driver.switch_to.window(driver.window_handles[0])
                                except Exception as e:
                                    print(f"Error al obtener detalles de '{titulo}': {str(e)}")
                                    
                                    # Aseguramos volver a la ventana principal si algo falla
                                    if len(driver.window_handles) > 1:
                                        driver.close()
                                        driver.switch_to.window(driver.window_handles[0])
                            
                            # Guardamos los datos en un diccionario con la estructura requerida
                            evento = {
                                'id_evento': id_evento,
                                'nombre_evento': titulo,
                                'categoria': categoria,
                                'discapacidad': discapacidad,
                                'ubicacion': ubicacion,
                                'costo': costo,
                                'edad_dirigida': edad_dirigida,
                                'min_integrantes': min_integrantes,
                                'modalidad': modalidad
                            }
                            
                            eventos.append(evento)
                            
                        except Exception as e:
                            print(f"Error al procesar elemento {i+1}: {str(e)}")
                    
                    # Si encontramos elementos con este selector, no seguimos probando
                    break
            except Exception as e:
                print(f"Error con selector '{selector}': {str(e)}")
        
        # Si no hemos encontrado eventos con los selectores anteriores, intentamos una aproximación más general
        if not elementos_encontrados:
            print("Intentando aproximación alternativa...")
            
            # Obtenemos todos los enlaces de la página
            enlaces = driver.find_elements(By.TAG_NAME, 'a')
            
            # Filtramos enlaces que parezcan ser de eventos
            enlaces_eventos = []
            for enlace in enlaces:
                try:
                    href = enlace.get_attribute('href')
                    texto = enlace.text.strip()
                    
                    # Si el enlace tiene href, texto, y parece ser un evento (no una categoría o navegación)
                    if href and texto and 'planinfantil.es/planes/' in href and not 'category' in href and not 'page' in href:
                        enlaces_eventos.append((texto, href))
                except:
                    continue
            
            print(f"Se encontraron {len(enlaces_eventos)} posibles enlaces a eventos")
            
            # Procesamos cada enlace
            for i, (titulo, url_evento) in enumerate(enlaces_eventos):
                try:
                    print(f"Procesando evento {i+1}: {titulo}")
                    
                    # Generamos un ID único
                    id_evento = str(uuid.uuid4())[:8]
                    
                    # Valores por defecto
                    descripcion = "No disponible"
                    categoria = categorizar_evento(titulo, descripcion)
                    discapacidad = "Ninguna"
                    ubicacion = "Madrid"
                    costo = "No especificado"
                    edad_dirigida = "No especificado"
                    min_integrantes = "No especificado"
                    modalidad = "No especificado"
                    
                    # Visitamos la página del evento para obtener más detalles
                    try:
                        # Abrimos la página del evento
                        driver.get(url_evento)
                        time.sleep(3)
                        
                        # Obtenemos todo el texto de la página
                        texto_detallado = driver.find_element(By.TAG_NAME, 'body').text
                        
                        # Actualizamos datos con información más detallada
                        categoria = categorizar_evento(titulo, texto_detallado)
                        discapacidad = determinar_discapacidad(titulo, texto_detallado)
                        costo = extraer_costo(texto_detallado)
                        edad_dirigida = determinar_edad(texto_detallado)
                        modalidad = determinar_modalidad(texto_detallado)
                        min_integrantes = determinar_min_integrantes(texto_detallado)
                        
                        # Intentamos encontrar la ubicación
                        for palabra_clave in ['ubicación', 'dirección', 'lugar', 'se celebra en']:
                            if palabra_clave in texto_detallado.lower():
                                indice = texto_detallado.lower().find(palabra_clave)
                                ubicacion_texto = texto_detallado[indice:indice+100]  # Tomamos 100 caracteres después
                                ubicacion = ubicacion_texto.split('\n')[0].strip()
                                break
                        
                        # Volvemos a la página principal
                        driver.get(url)
                        time.sleep(2)
                        
                    except Exception as e:
                        print(f"Error al obtener detalles de '{titulo}': {str(e)}")
                        
                        # Volvemos a la página principal
                        driver.get(url)
                        time.sleep(2)
                    
                    # Guardamos los datos en un diccionario con la estructura requerida
                    evento = {
                        'id_evento': id_evento,
                        'nombre_evento': titulo,
                        'categoria': categoria,
                        'discapacidad': discapacidad,
                        'ubicacion': ubicacion,
                        'costo': costo,
                        'edad_dirigida': edad_dirigida,
                        'min_integrantes': min_integrantes,
                        'modalidad': modalidad
                    }
                    
                    eventos.append(evento)
                    
                except Exception as e:
                    print(f"Error al procesar enlace {i+1}: {str(e)}")
        
        # Creamos un DataFrame con la información estructurada
        df = pd.DataFrame(eventos)
        
        if not df.empty:
            # Eliminamos posibles duplicados
            df = df.drop_duplicates(subset=['nombre_evento'])
            
            # Guardamos los datos en un archivo CSV
            df.to_csv('planes_infantiles_selenium.csv', index=False, encoding='utf-8-sig')
            
            print(f"Se han extraído y estructurado {len(df)} eventos y se han guardado en 'planes_infantiles_selenium.csv'")
        else:
            print("No se encontraron eventos para extraer")
        
        return df
        
    except Exception as e:
        print(f"Error general durante el scraping: {str(e)}")
        return None
        
    finally:
        # Cerramos el driver
        if driver:
            driver.quit()

if __name__ == "__main__":
    print("Iniciando web scraping con Selenium...")
    df = scrape_planinfantil_selenium()
    
    if df is not None and not df.empty:
        print("\nPrimeros 5 eventos extraídos:")
        print(df.head())
    else:
        print("\nNo se pudo extraer información de eventos")

Iniciando web scraping con Selenium...
Accediendo a https://www.planinfantil.es/planes/...
Título de la página: Planes con niños en Madrid - planinfantil.es
HTML guardado en 'pagina_selenium.html'
Buscando eventos en la página...
Se encontraron 1 elementos con el selector 'article'
Evento 1: PARQUES - URL: https://www.planinfantil.es/planes/parques/
Se han extraído y estructurado 1 eventos y se han guardado en 'planes_infantiles_selenium.csv'

Primeros 5 eventos extraídos:
  id_evento nombre_evento categoria discapacidad ubicacion            costo  \
0  6bd28bb9       PARQUES  Talleres      Ninguna    Madrid  No especificado   

  edad_dirigida min_integrantes modalidad  
0     3-12 años               1  Exterior  


In [8]:
df

,id_evento,nombre_evento,categoria,discapacidad,ubicacion,costo,edad_dirigida,min_integrantes,modalidad
0,6bd28bb9,PARQUES,Talleres,Ninguna,Madrid,No especificado,3-12 años,1,Exterior


In [12]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

def scrape_planinfantil_selenium():
    """
    Función para hacer web scraping de la página https://www.planinfantil.es/planes/
    utilizando Selenium para cargar contenido dinámico
    """
    url = "https://www.planinfantil.es/planes/"
    
    # Inicializamos el driver
    driver = inicializar_driver()
    if not driver:
        print("No se pudo inicializar el driver de Selenium.")
        return None
    
    try:
        # Abrimos la página
        driver.get(url)
        
        # Esperamos a que los eventos estén presentes
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.elementor-posts-container article'))
        )
        
        # Lista para almacenar los eventos
        eventos_all = []
        
        # Extraemos eventos de la página actual
        elementos = driver.find_elements(By.CSS_SELECTOR, '.elementor-posts-container article')
        print(f"Se encontraron {len(elementos)} eventos en esta página.")
        
        # Procesamos cada evento
        for i, elemento in enumerate(elementos):
            # Aquí va el código para extraer título, descripción, etc.
            pass
        
        # Intentamos obtener más páginas
        while True:
            try:
                siguiente_pagina = driver.find_element(By.XPATH, '//a[@class="next"]')
                siguiente_pagina.click()
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.elementor-posts-container article'))
                )
                elementos = driver.find_elements(By.CSS_SELECTOR, '.elementor-posts-container article')
                print(f"Se encontraron {len(elementos)} eventos en esta página.")
                
                # Procesamos los eventos de la nueva página
                for i, elemento in enumerate(elementos):
                    # Aquí va el código para extraer título, descripción, etc.
                    pass
                
            except Exception as e:
                print("No hay más páginas para procesar.")
                break  # Salimos del bucle si no hay siguiente página
        
        # Creamos el DataFrame y guardamos los eventos
        df = pd.DataFrame(eventos_all)
        
        if not df.empty:
            df = df.drop_duplicates(subset=['nombre_evento'])
            df.to_csv('planes_infantiles_selenium.csv', index=False, encoding='utf-8-sig')
            print(f"Se han extraído {len(df)} eventos.")
        else:
            print("No se encontraron eventos para extraer")
        
        return df
        
    except Exception as e:
        print(f"Error general durante el scraping: {str(e)}")
        return None
    finally:
        if driver:
            driver.quit()


In [14]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

def scrape_planinfantil_selenium():
    """
    Función para hacer web scraping de la página https://www.planinfantil.es/planes/
    utilizando Selenium para cargar contenido dinámico
    """
    url = "https://www.planinfantil.es/planes/"
    
    # Inicializamos el driver
    driver = inicializar_driver()
    if not driver:
        print("No se pudo inicializar el driver de Selenium.")
        return None
    
    try:
        # Abrimos la página
        driver.get(url)
        
        # Esperamos a que los eventos estén presentes
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.elementor-posts-container article'))
        )
        
        # Lista para almacenar los eventos
        eventos_all = []
        
        # Extraemos eventos de la página actual
        elementos = driver.find_elements(By.CSS_SELECTOR, '.elementor-posts-container article')
        print(f"Se encontraron {len(elementos)} eventos en esta página.")
        
        # Procesamos cada evento
        for i, elemento in enumerate(elementos):
            # Aquí va el código para extraer título, descripción, etc.
            pass
        
        # Intentamos obtener más páginas
        while True:
            try:
                siguiente_pagina = driver.find_element(By.XPATH, '//a[@class="next"]')
                siguiente_pagina.click()
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.elementor-posts-container article'))
                )
                elementos = driver.find_elements(By.CSS_SELECTOR, '.elementor-posts-container article')
                print(f"Se encontraron {len(elementos)} eventos en esta página.")
                
                # Procesamos los eventos de la nueva página
                for i, elemento in enumerate(elementos):
                    # Aquí va el código para extraer título, descripción, etc.
                    pass
                
            except Exception as e:
                print("No hay más páginas para procesar.")
                break  # Salimos del bucle si no hay siguiente página
        
        # Creamos el DataFrame y guardamos los eventos
        df = pd.DataFrame(eventos_all)
        
        if not df.empty:
            df = df.drop_duplicates(subset=['nombre_evento'])
            df.to_csv('planes_infantiles_selenium.csv', index=False, encoding='utf-8-sig')
            print(f"Se han extraído {len(df)} eventos.")
        else:
            print("No se encontraron eventos para extraer")
        
        return df
        
    except Exception as e:
        print(f"Error general durante el scraping: {str(e)}")
        return None
    finally:
        if driver:
            driver.quit()


In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import uuid
import pandas as pd

def inicializar_driver():
    """
    Inicializa y configura el driver de Selenium
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Ejecutar en modo headless (sin interfaz gráfica)
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        return driver
    except Exception as e:
        print(f"Error al inicializar el driver: {str(e)}")
        return None

def scrape_planinfantil_selenium():
    """
    Función para hacer web scraping de la página https://www.planinfantil.es/planes/
    utilizando Selenium para cargar contenido dinámico
    """
    driver = inicializar_driver()
    if not driver:
        print("No se pudo inicializar el driver de Selenium.")
        return None
    
    url = "https://www.planinfantil.es/planes/"
    
    try:
        # Abrimos la página
        driver.get(url)
        
        # Esperamos a que la página cargue completamente
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.elementor-posts-container'))
        )
        
        # Verificamos si la página se ha cargado correctamente
        print(f"Título de la página: {driver.title}")
        
        # Intentamos extraer los elementos de la página
        elementos = driver.find_elements(By.CSS_SELECTOR, '.elementor-posts-container article')
        print(f"Se encontraron {len(elementos)} eventos en esta página.")
        
        eventos_all = []

        # Procesamos los elementos
        for i, elemento in enumerate(elementos):
            try:
                titulo = None
                for selector_titulo in ['h3', 'h2', 'h1', '.elementor-post__title']:
                    try:
                        titulo_elem = elemento.find_element(By.CSS_SELECTOR, selector_titulo)
                        titulo = titulo_elem.text.strip()
                        if titulo:
                            break
                    except Exception as e:
                        print(f"Error al extraer título: {str(e)}")
                
                if not titulo:
                    continue
                
                descripcion = "No disponible"
                for selector_desc in ['.elementor-post__excerpt', 'p', '.elementor-post__text']:
                    try:
                        desc_elem = elemento.find_element(By.CSS_SELECTOR, selector_desc)
                        descripcion = desc_elem.text.strip()
                        if descripcion:
                            break
                    except Exception as e:
                        print(f"Error al extraer descripción: {str(e)}")
                
                url_evento = None
                try:
                    link = elemento.find_element(By.TAG_NAME, 'a')
                    url_evento = link.get_attribute('href')
                except Exception as e:
                    print(f"Error al obtener URL del evento: {str(e)}")
                
                # Generamos un ID único para el evento
                id_evento = str(uuid.uuid4())[:8]

                evento = {
                    'id_evento': id_evento,
                    'nombre_evento': titulo,
                    'categoria': 'No especificada',  # Puedes agregar la lógica para categorizar
                    'discapacidad': 'Ninguna',  # Lógica para discapacidad
                    'ubicacion': 'Madrid',  # Lógica para ubicación
                    'costo': 'No especificado',  # Extraer el costo si es posible
                    'edad_dirigida': 'No especificado',  # Lógica para edad dirigida
                    'min_integrantes': 'No especificado',  # Lógica para determinar mínimo de integrantes
                    'modalidad': 'No especificada',  # Lógica para modalidad
                }

                eventos_all.append(evento)
            except Exception as e:
                print(f"Error al procesar el evento {i+1}: {str(e)}")

        # Convertimos los eventos en un DataFrame y lo guardamos en un archivo CSV
        df = pd.DataFrame(eventos_all)

        if not df.empty:
            df = df.drop_duplicates(subset=['nombre_evento'])
            df.to_csv('planes_infantiles_selenium.csv', index=False, encoding='utf-8-sig')
            print(f"Se han extraído {len(df)} eventos y se han guardado en 'planes_infantiles_selenium.csv'")
        else:
            print("No se encontraron eventos para extraer")
        
        return df

    except Exception as e:
        pridddfdfnt(f"Error general durante el scraping: {str(e)}")
        return None
    finally:
dfddfffm miver:
            di

# Ejecutar la función de scraping
if __name__ == "__main__":
    df = scrape_planinfantil_selenium()
    if df is not None and not df.empty:
        print("\nPrimeros 5 eventos extraídos:")
        print(df.head())
    else:
        print("\nNo se pudo extraer información de eventos")


Error general durante el scraping: Message: 
Stacktrace:
0   chromedriver                        0x00000001059242c8 chromedriver + 6197960
1   chromedriver                        0x000000010591b8ea chromedriver + 6162666
2   chromedriver                        0x00000001053a0de0 chromedriver + 417248
3   chromedriver                        0x00000001053f2797 chromedriver + 751511
4   chromedriver                        0x00000001053f29b1 chromedriver + 752049
5   chromedriver                        0x00000001054429b4 chromedriver + 1079732
6   chromedriver                        0x00000001054189ed chromedriver + 907757
7   chromedriver                        0x000000010543fcdb chromedriver + 1068251
8   chromedriver                        0x0000000105418793 chromedriver + 907155
9   chromedriver                        0x00000001053e4b25 chromedriver + 695077
10  chromedriver                        0x00000001053e5791 chromedriver + 698257
11  chromedriver                        0x000000

In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# Configurar opciones de Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecutar sin interfaz gráfica si es necesario

# Inicializar el driver con webdriver_manager (automáticamente gestionará la versión de ChromeDriver)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Ahora puedes usar el driver para realizar scraping
driver.get('https://www.planinfantil.es/planes/')  # Ejemplo de página

# Realiza el scraping aquí...
print(driver.title)

# Cerrar el driver cuando termine
driver.quit()


Planes con niños en Madrid - planinfantil.es


In [23]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Espera a que el contenedor de los artículos esté cargado
WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.elementor-posts-container article'))
)

# Extraemos los artículos de la página
elementos = driver.find_elements(By.CSS_SELECTOR, '.elementor-posts-container article')

print(f"Se encontraron {len(elementos)} eventos.")


MaxRetryError: HTTPConnectionPool(host='localhost', port=50196): Max retries exceeded with url: /session/6f835628cfe62ccd3a5e2e87651603ef/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11e1ee450>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configurar opciones de Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecutar sin interfaz gráfica si es necesario

# Inicializar el driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Acceder a la página
driver.get("https://www.planinfantil.es/planes/")

# Esperar a que los artículos estén cargados
WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.elementor-posts-container article'))
)

# Extraer los eventos
elementos = driver.find_elements(By.CSS_SELECTOR, '.elementor-posts-container article')
print(f"Se encontraron {len(elementos)} eventos.")

# Imprimir el título de la página
print("Título de la página:", driver.title)

# Cerrar el driver
driver.quit()


TimeoutException: Message: 


In [26]:
import requests
from bs4 import BeautifulSoup

# URL de la página de eventos
url = 'https://www.planinfantil.es/planes/'

# Realizamos la solicitud HTTP a la página
response = requests.get(url)

# Verificamos que la respuesta sea exitosa
if response.status_code == 200:
    # Parseamos el contenido de la página con BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Imprimimos el título de la página para verificar que hemos cargado correctamente
    print(f"Título de la página: {soup.title.string}")
    
    # Buscar los elementos correctos (cambia el selector según la estructura)
    eventos = soup.select('.panel-grid article')  # Usa el selector adecuado que encuentres en la página

    # Verificamos cuántos eventos hemos encontrado
    print(f"Se encontraron {len(eventos)} eventos.")
    
    # Extraer detalles de los eventos
    for i, evento in enumerate(eventos):
        try:
            # Extraemos el título del evento
            titulo = evento.find('h2').text.strip() if evento.find('h2') else 'Sin título'
            
            # Extraemos la descripción del evento (ajusta según lo que encuentres en el HTML)
            descripcion = evento.find('p').text.strip() if evento.find('p') else 'Sin descripción'
            
            # Imprimir los resultados
            print(f"Evento {i + 1}:")
            print(f"Título: {titulo}")
            print(f"Descripción: {descripcion}")
            print("-" * 50)
        
        except Exception as e:
            print(f"Error al procesar el evento {i + 1}: {str(e)}")
else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")


Título de la página: Planes con niños en Madrid - planinfantil.es
Se encontraron 0 eventos.
